
## Data Science Summit - Python Challenge : 8-Queens Puzzle
 
Description

The eight queens puzzle is the problem of placing eight chess queens on an 8×8 chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. The eight queens puzzle is an example of the more general n queens problem of placing n non-attacking queens on an n×n chessboard. (Source : https://en.wikipedia.org/wiki/Eight_queens_puzzle )

Challenge

The challenge is to generate one right sequence through Genetic Programming. The sequence has to be 8 numbers between 0 to 7. Each number represents the positions the Queens can be placed. Each number refers to the row number in the specific column

0 3 4 5 6 1 2 4

· 0 is the row number in the column 0 where the Queen can be placed

· 3 is the row number in the column 1 where the Queen can be placed

Author : Santosh Singh Rajput 

July 2020

In [4]:
import random
from numpy.random import choice
import pandas as pd
import threading 
import _thread
import sys

global correct_sequence,return_code
return_code="0"

def main():
    solution_list=[]
    global return_code,correct_sequence,gen_num
    
    ##### Genetic Algorithm variables  #####
    chess_size = 8
    mutationRate = 0.01
    totalPopulation = 150
    crossOver = 0.5
    generationCount = 1000
    ########################################
    
    # Calling the class method solve_nqueens_puzzle() and storing the return value in solution_list
    solution_list=NQueens_puzzle(chess_size).solve_nqueens_puzzle() 

    for i in range(len(solution_list)):
            thread_name=''.join(solution_list[i])
            thread_instance = threading.Thread(target=preprocessing, args=(chess_size,thread_name,mutationRate,totalPopulation,crossOver,generationCount,)) 
            thread_instance.start()
    print_str="\n"+" ######## "+ correct_sequence +" is one of the correct sequence of Queens. It is generated first in generation no :"+ gen_num +" ######## "
    print(print_str)
    


def inital_population(totalPopulation, chess_size, target, secure_random, alpha_list):
    populationData = []
    fitnessData = []
    for outloop in range(totalPopulation):
        randomData = []
        fitnessScore = 0
        for inloop in range(chess_size):
            selectedData = secure_random.choice(alpha_list)
            if (selectedData == target[inloop]):
                fitnessScore = fitnessScore + 1
            randomData.append(selectedData)
        populationData.append(randomData)
        fitnessData.append(fitnessScore)
    return (populationData, fitnessData)



def preprocessing(chess_size, thread_name, mutationRate, totalPopulation, crossOver, generationCount):
    global return_code, gen_num

    secure_random = random.SystemRandom()
    target = thread_name
    alpha_list = [chr(x) for x in range(ord('0'), ord('7') + 1)]
    populationData = []
    fitnessData = []
    populationData, fitnessData = inital_population(totalPopulation, chess_size, target, secure_random, alpha_list)
    probabilityDist = []
    for outloop in range(totalPopulation):
        probabilityDist.append(fitnessData[outloop] / len(target))
    probDataFrame = pd.DataFrame(
        {'String': populationData, 'FitnessScore': fitnessData, 'Probability': probabilityDist})
    probDataFrame = probDataFrame.sort_values(['Probability'], ascending=False)
    probDataFrame = probDataFrame.reset_index(drop=True)
    probDataFrame.head()
    crossover_n_mutation(alpha_list, crossOver, target, generationCount, probDataFrame, thread_name, secure_random,
                         populationData, probabilityDist, fitnessData)

        
        
def crossover_n_mutation(alpha_list, crossOver, target, generationCount, probDataFrame, thread_name, secure_random,
                         populationData, probabilityDist, fitnessData):
    global return_code, correct_sequence, gen_num

    crossOverPoint = int(crossOver * len(target))
    # print("\n")
    for loop in range(generationCount):
        if return_code == "1":
            sys.exit(0)
        else:
            draw = []
            draw.append(probDataFrame[0:1]["String"].values[0])
            draw.append(probDataFrame[1:2]["String"].values[0])
            if (getFitnessScore(draw[0], target) == len(target) | getFitnessScore(draw[1], target) == len(target)):
                print_str = ""
                print_str0 = ''.join([elem for elem in draw[0]])
                print_str1 = ''.join([elem for elem in draw[1]])
                print(print_str)

                return_code = "1"
                correct_sequence = thread_name

                gen_num = str(loop)
                sys.exit(0)
                break
            child1 = draw[0][0:crossOverPoint] + draw[1][crossOverPoint:]
            child2 = draw[1][0:crossOverPoint] + draw[0][crossOverPoint:]
            child1[random.randint(0, len(target) - 1)] = secure_random.choice(alpha_list)
            child2[random.randint(0, len(target) - 1)] = secure_random.choice(alpha_list)
            populationData.append(child1)
            populationData.append(child2)
            fitnessData = []
            totalPopulation = len(populationData)
            for outloop in range(totalPopulation):
                fitnessScore = getFitnessScore(populationData[outloop], target)
                fitnessData.append(fitnessScore)
            probabilityDist = []
            for outloop in range(totalPopulation):
                probabilityDist.append(fitnessData[outloop] / sum(fitnessData))
            probDataFrame = pd.DataFrame(
                {'String': populationData, 'FitnessScore': fitnessData, 'Probability': probabilityDist})
            probDataFrame = probDataFrame.sort_values(['Probability'], ascending=False)
            probDataFrame = probDataFrame.reset_index(drop=True)
            if return_code == "1":
                sys.exit(0)
            else:

                a = ""
                output_string = "\n" + 'Thread ' + str(thread_name) + ' Generation ' + str(
                    loop) + ' ' + '|Offspring1 :' + ''.join(elem for elem in child1) + '|FS of Offspring1: ' + str(
                    getFitnessScore(child1, target)) + '|Offspring2 :' + ''.join(
                    elem for elem in child2) + '|FS of Offspring2 :' + str(getFitnessScore(child2, target))
                print(output_string)

                
            
def getFitnessScore(data,target):

          data = ''.join([elem for elem in data])
          fitnessScore = 0
          for inloop in range(len(target)):
                if (data[inloop] == target[inloop]):
                        fitnessScore = fitnessScore + 1
          return fitnessScore



def maxProb(probabilityDist):
        probabilityList = [f for f in set(probabilityDist)]
        return (probabilityList[len(probabilityList)-2])



def viewElement(data):
          data = ''.join([elem for elem in data])
          return data        

        

class NQueens_puzzle:
    #This is the class to generate all the valid solutions for the N Queens puzzle
    def __init__(self,size):
        # Chess Board Size and valid solutions count
        self.size = size
        self.total_solutions_count = 0


    def solve_nqueens_puzzle(self):
        #Solve the N Queens puzzle and return the number of solutions.

        final_list=[]
        positions = [-1] * self.size
        self.place_queen(positions, 0,final_list)
        return final_list


    def place_queen(self, positions, current_row,final_list):
        
        # This class method places a queen on the row being checked(current_row) by ignoring the places under attack
        # due to already filled rows and selecting the first safe position found in the column for the current row.
        # If a valid place is found, then the function calls itself trying to place a queen
        # on the next row until all N queens are placed on the NxN board.
        
        if current_row == self.size:
            
            self.prep_final_list(positions,final_list)
            self.total_solutions_count += 1
            #print(f"Solution Count :: {self.total_solutions_count}")
            #print(f"Solution ::  {positions}")
        else:
            # Call the check_pos class method to retrieve the safe column position for the current row
            # which is not under attack from already filled positions.
            # Place the Queen in the safe column and iterate again for the next row Queen position.
            for column in range(self.size):
                if self.check_pos(positions, current_row, column):
                    positions[current_row] = column
                    self.place_queen(positions, current_row + 1,final_list)
        


    def check_pos(self, positions, filled_rows, column):
        # Check and return False if the current column is under attack from any of filled Queens.
        # Return True if column is not under attack.
        for i in range(filled_rows):
            if positions[i] == column or \
                positions[i] - i == column - filled_rows or \
                positions[i] + i == column + filled_rows:

                return False
        return True

    def prep_final_list(self, positions,final_list):
        # Prepare the final list by appending each solution set
        
        current_list=[]
        for row in range(self.size):
            line = ""
            for column in range(self.size):
                if positions[row] == column:
                    current_list.append(str(column))
        final_list.append(current_list)



if __name__ == "__main__":

    main()


Thread 05726314 Generation 0 |Offspring1 :05446174|FS of Offspring1: 4|Offspring2 :71150316|FS of Offspring2 :2
Thread 06357142 Generation 0 |Offspring1 :75733122|FS of Offspring1: 2|Offspring2 :56577142|FS of Offspring2 :5

Thread 04752613 Generation 0 |Offspring1 :06116613|FS of Offspring1: 4|Offspring2 :20672425|FS of Offspring2 :1


Thread 05726314 Generation 1 |Offspring1 :05446312|FS of Offspring1: 5|Offspring2 :05446144|FS of Offspring2 :4

Thread 04752613 Generation 1 |Offspring1 :06112032|FS of Offspring1: 2|Offspring2 :64006113|FS of Offspring2 :3

Thread 05726314 Generation 2 |Offspring1 :05476144|FS of Offspring1: 4|Offspring2 :05446352|FS of Offspring2 :4

Thread 06357142 Generation 1 |Offspring1 :56570142|FS of Offspring1: 4|Offspring2 :76737112|FS of Offspring2 :4

Thread 05726314 Generation 3 |Offspring1 :05446052|FS of Offspring1: 3|Offspring2 :05440312|FS of Offspring2 :4

Thread 04752613 Generation 2 |Offspring1 :06116153|FS of Offspring1: 2|Offspring2 :64006613|FS 


Thread 20647135 Generation 4 |Offspring1 :20627735|FS of Offspring1: 6|Offspring2 :20727735|FS of Offspring2 :5

Thread 15720364 Generation 10 |Offspring1 :14325374|FS of Offspring1: 4|Offspring2 :10520374|FS of Offspring2 :5
Thread 14630752 Generation 7 |Offspring1 :11333352|FS of Offspring1: 4|Offspring2 :11230355|FS of Offspring2 :4

Thread 15063724 Generation 3 |Offspring1 :15003223|FS of Offspring1: 5|Offspring2 :15163427|FS of Offspring2 :5


Thread 24730615 Generation 0 |Offspring1 :24332046|FS of Offspring1: 3|Offspring2 :25731565|FS of Offspring2 :4

Thread 20647135 Generation 5 |Offspring1 :20627737|FS of Offspring1: 5|Offspring2 :25627735|FS of Offspring2 :5

Thread 06471352 Generation 5 |Offspring1 :06106212|FS of Offspring1: 3|Offspring2 :06431446|FS of Offspring2 :4

Thread 24603175 Generation 0 |Offspring1 :71601373|FS of Offspring1: 3|Offspring2 :27673371|FS of Offspring2 :4
Thread 24175360 Generation 1 |Offspring1 :34134160|FS of Offspring1: 4|Offspring2 :24016360|FS 


Thread 24170635 Generation 3 |Offspring1 :14170533|FS of Offspring1: 5|Offspring2 :14170635|FS of Offspring2 :7

Thread 25164073 Generation 4 |Offspring1 :06164064|FS of Offspring1: 4|Offspring2 :16123013|FS of Offspring2 :3

Thread 25704613 Generation 1 |Offspring1 :76734354|FS of Offspring1: 2|Offspring2 :75774211|FS of Offspring2 :4

Thread 25317460 Generation 2 |Offspring1 :25324420|FS of Offspring1: 5|Offspring2 :27714420|FS of Offspring2 :4

Thread 06357142 Generation 11 |Offspring1 :06471142|FS of Offspring1: 5|Offspring2 :06517142|FS of Offspring2 :6

Thread 14602753 Generation 10 |Offspring1 :11616753|FS of Offspring1: 5|Offspring2 :16006756|FS of Offspring2 :4

Thread 14630752 Generation 11 |Offspring1 :11230252|FS of Offspring1: 5|Offspring2 :11230352|FS of Offspring2 :5
Thread 26174035 Generation 0 |Offspring1 :41370072|FS of Offspring1: 2|Offspring2 :16272025|FS of Offspring2 :4
Thread 16470352 Generation 4 |Offspring1 :13473351|FS of Offspring1: 5|Offspring2 :10423353|FS


 ######## 20647135 is one of the correct sequence of Queens. It is generated first in generation no :14 ######## 
